This notebook will show you how to organize in 2D a set of documents/articles/posts so that articles with similar content are grouped near to each other. The example I am using is a set of Wikipedia articles of [Political ideologies](https://en.wikipedia.org/wiki/List_of_political_ideologies), but in principle it can be used for any set of documents. 

The result of this notebook [can be viewed live here](https://www.genekogan.com/works/wiki-tSNE/).

### Procedure

The pipeline consists of two steps.

1) Convert all of the articles to a [tf-idf matrix](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

tf-idf stands for "term frequency inverse document frequency" and is commonly used in natural language processing applications dealing with large collections of documents. A tf-idf matrix is an $n * m$ sparse matrix consisting of $n$ rows, corresponding to our $n$ documents, and $m$ columns, corresponding to the $m$ unique "terms" (usually just words but can be n-grams or other kinds of tokens) that appear in the entire corpus.

Each entry in the matrix, $tfidf(i,j)$ can be interpreted as the "relative importance" of term $j$ to document $i$.  It is calculated as

$$tfidf(i,j) = tf(i,j)*idf(i,j)$$

$tf(i, j)$ is the "term frequency," i.e. the percentage of terms in document $i$ which are term $j$. For example, in the document "the cat in the hat", the term "the" has a $tf$ of (2 / 5) = 0.4. Thus $tf$ is high when the term is frequently found in the document.

$idf(i, j)$, not to be confused with [this IDF](https://twitter.com/idfspokesperson/status/547144026445471744) is the "inverse document frequency." It is given by:

$$idf(i, j) = log(\frac{N}{n_j})$$

where $N$ is the number of documents in the corpus and $n_j$ is the number of documents which contain term $j$. When $n_j$ is high, this value shrinks towards 0. This happens when the term frequently appears in many or all documents, thus common terms like "the", "a", "it", etc will have a low $idf$ score because they appear in most documents. Conversely, when the term rarely appears in documents ($n_j$ is low), then $idf$ score will be high. These tend to be special or topic-specific words which appear in few of the documents.

So intuitively, the $tfidf$ score for a term in a document goes higher if the term appears frequently in the document and appears infrequently in other documents (so that term is important to that document).

2) This gives you a high-dimensional matrix of n documents which can be reduced to 2 dimensions using the [t-SNE](https://lvdmaaten.github.io/tsne/) dimensionality reduction technique. A better description of how t-SNE works can be found in the link.

### Installation

You need [nltk](http://www.nltk.org/install.html) and [scikit-learn](http://scikit-learn.org/) to run most of this notebook.

    pip install -U nltk
    pip install -U scikit-learn

Also, if you don't already have [numpy](http://www.numpy.org/), you should install it as well (it's only used to normalize the data later). 

    pip install numpy

Additionally, if you are following this example and wish to extract articles from Wikipedia, you need the python [wikipedia](https://pypi.python.org/pypi/wikipedia/) library. 

    pip install wikipedia
  

First make sure all these imports work (minus wikipedia if you intend to use another corpus). We will assume wikipedia from here.

In [1]:
import string
import os
import time
import cPickle as pickle
import json
import wikipedia
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD

In this example, we are going to cluster all of the links found in the Wikipedia page "[List of political ideologies](https://en.wikipedia.org/wiki/List_of_political_ideologies)." First, we will open the page in main, then add all of the raw text of the articles into a dictionary called token_dict.

In [2]:
main = wikipedia.page('List of political ideologies')
token_dict = {}
for i, article in enumerate(main.links):
    if article not in token_dict:
        time.sleep(5)  # helps to avoid hangups. Ctrl-C in case you get stuck on one
        if i%20==0:
            print "getting text for article %d/%d : %s"%(i, len(main.links), article)
        try:
            text = wikipedia.page(article)
            token_dict[article] = text.content
        except:
            print " ==> error processing "+article


getting text for article 0/410 : Absolute monarchy
getting text for article 20/410 : Anti-revisionism
getting text for article 40/410 : Bolivarianism
getting text for article 60/410 : Christian anarchism
getting text for article 80/410 : Confederation
getting text for article 100/410 : Cultural liberalism
getting text for article 120/410 : Empire
getting text for article 140/410 : Fundamentalism
getting text for article 160/410 : Ho Chi Minh Thought
getting text for article 180/410 : Insurrectionary anarchism
getting text for article 200/410 : Kemalism
getting text for article 220/410 : Libertarian Marxism
getting text for article 240/410 : Masculism
getting text for article 260/410 : National Unity government
getting text for article 280/410 : One-party state
 ==> error processing Panarchism
getting text for article 300/410 : Party platform
getting text for article 320/410 : Progressivism
getting text for article 340/410 : Republic
 ==> error processing Rexism
getting text for article

/Users/gene/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


You may find it useful to save the dictionary so as to not have to re-download all the articles later.

In [3]:
pickle.dump(token_dict, open("token_dict_political_ideologies.p", "wb" ))

# later you can retrieve it like this:
#token_dict = pickle.load(open("token_dict_political_ideologies.p", "r" ))

Next, we will evaluate the tf-idf matrix of our collection of articles. First we need to define a tokenizer function which will properly convert all the raw text of the articles into a vector of tokens (our terms).

The function `tokenize` will take the raw text, convert it to lower-case, strip out punctuation and special characters, remove all stop words (common words in english, e.g. "the", "and", "i", etc), stem the remaining words ([using Porter stemmer](https://en.wikipedia.org/wiki/Stemming)) to unify tokens in different parts-of-speech (e.g. "run", "running", "ran" --> "run"), and output what's left as a vector.

From there we run the tfidf vectorizer which will return us the resulting tfidf matrix. Then we use [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition) to reduce the dimensionality before t-SNE (this is optional, but some sources recommend it). 

Note, calculating the tfidf matrix can take a while.

In [4]:
def tokenize(text):
    text = text.lower() # lower case
    for e in set(string.punctuation+'\n'+'\t'): # remove punctuation and line breaks/tabs
        text = text.replace(e, ' ')	
    for i in range(0,10):	# remove double spaces
        text = text.replace('  ', ' ')
    text = text.translate(string.punctuation)  # punctuation
    tokens = nltk.word_tokenize(text)
    text = [w for w in tokens if not w in stopwords.words('english')] # stopwords
    stems = []
    for item in tokens: # stem
        stems.append(PorterStemmer().stem(item))
    return stems

# calculate tfidf (might take a while)
print "calculating tf-idf"
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())
print "reducing tf-idf to 500 dim"
tfs_reduced = TruncatedSVD(n_components=500, random_state=0).fit_transform(tfs)
print "done"

calculating tf-idf
reducing tf-idf to 500 dim
done


Let's quickly see what we have computed. Your exact results may vary.  

`tfs` is our raw tf-idf matrix.  We can print it and see the values. For example, tfs(0, 20000) is the tfidf score for document 0, term 20000. We can query to find out what this term is. 

In [9]:
print tfs
print "term 20000 = \"%s\""%tfidf.get_feature_names()[20000]

  (0, 30109)	0.00883237481975
  (0, 1099)	0.00981579227116
  (0, 18585)	0.0173205353806
  (0, 19820)	0.0185190161963
  (0, 26580)	0.00989461873953
  (0, 1125)	0.0100588730717
  (0, 333)	0.0137618116339
  (0, 6688)	0.0117129951469
  (0, 11325)	0.0121933714241
  (0, 16927)	0.00618028973016
  (0, 6393)	0.00465224856461
  (0, 1838)	0.00769019698985
  (0, 18429)	0.0140732380778
  (0, 20813)	0.00765254275598
  (0, 18045)	0.00659453758079
  (0, 2594)	0.0185190161963
  (0, 10433)	0.00801005658467
  (0, 2)	0.0145344340567
  (0, 4625)	0.0158535772335
  (0, 16236)	0.00906432258494
  (0, 22410)	0.0174457751699
  (0, 11955)	0.00425924948829
  (0, 1728)	0.00906432258494
  (0, 26871)	0.0164701997093
  (0, 24598)	0.00476275928191
  :	:
  (385, 29828)	0.0587711670585
  (385, 7166)	0.00227550091861
  (385, 9263)	0.00985853068481
  (385, 4122)	0.00326453542147
  (385, 15107)	0.0025366095366
  (385, 14047)	0.00531973808189
  (385, 15833)	0.0102509776583
  (385, 28532)	0.089936916532
  (385, 20637)	0.00357

Let's also look at tfs_reduced, which comes from our truncated SVD. note, we requested 500 dims, but it returned 408 because that's how many documents we managed to download.

In [10]:
print "size of TSVD: "+str(tfs_reduced.shape)
print tfs_reduced

size of TSVD: (386, 386)
[[  6.29518411e-01   1.59304474e-01  -3.79648006e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.02527409e-01   1.37833192e-01  -2.74399646e-01 ...,   4.69879382e-33
   -1.86184512e-32  -4.80886726e-33]
 [  1.65900711e-01  -6.24006739e-02   4.08613097e-02 ...,  -4.31602180e-32
    1.56216425e-32   9.80053863e-33]
 ..., 
 [  4.82534999e-01  -3.44946433e-02   1.23875202e-01 ...,   1.55345624e-17
   -2.03104533e-18   8.05032871e-19]
 [  1.01249033e-01   8.90437471e-03   1.95603819e-02 ...,   1.29216830e-32
    1.68020169e-32  -1.77168794e-33]
 [  2.54122504e-01  -3.39835124e-02   3.63681354e-02 ...,  -3.99397475e-32
    3.17875996e-33   1.06301276e-32]]


Finally, we run t-SNE on the result, normalize the t-SNE results (for convenience), and save the final coordinates to a json file.

Notice that we run it on `tfs_reduced`. You can also run it on the original tf-idf matrix (results should be similar). Make sure to remember to make it a dense matrix, i.e. `tfs.todense()`.

You may also want to play a bit with the parameters, like the size of the SVD reduction (probably minor effect) or the perplexity of the t-SNE.

At this point, the 2d normalized t-SNE points have been saved to a JSON file, along with the corresponding names of the articles. A nice way to visualize it is to display the terms in the browser. I've made a p5.js sketch which does this.

In [11]:
#model = TSNE(n_components=2, perplexity=50, verbose=2).fit_transform(tfs.todense())
model = TSNE(n_components=2, perplexity=50, verbose=2).fit_transform(tfs_reduced)

# save to json file
x_axis=model[:,0]
y_axis=model[:,1]
x_norm = (x_axis-np.min(x_axis)) / (np.max(x_axis) - np.min(x_axis))
y_norm = (y_axis-np.min(y_axis)) / (np.max(y_axis) - np.min(y_axis))
data = {"x":x_norm.tolist(), "y":y_norm.tolist(), "names":token_dict.keys()}
with open('data_political_ideologies.json', 'w') as outfile:
    json.dump(data, outfile)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 386 / 386
[t-SNE] Mean sigma: 0.310351
[t-SNE] Iteration 10: error = 21.1211239, gradient norm = 0.0656523
[t-SNE] Iteration 20: error = 19.1175275, gradient norm = 0.0897504
[t-SNE] Iteration 30: error = 18.6889811, gradient norm = 0.0844317
[t-SNE] Iteration 32: did not make any progress during the last 30 episodes. Finished.
[t-SNE] Iteration 40: error = 19.3492974, gradient norm = 0.0699504
[t-SNE] Iteration 50: error = 18.6104129, gradient norm = 0.0703584
[t-SNE] Iteration 60: error = 18.9324655, gradient norm = 0.0649455
[t-SNE] Iteration 66: did not make any progress during the last 30 episodes. Finished.
[t-SNE] Error after 66 iterations with early exaggeration: 18.050238
[t-SNE] Iteration 70: error = 2.5665829, gradient norm = 0.0151954
[t-SNE] Iteration 80: error = 1.8550274, gradient norm = 0.0193772
[t-SNE] Iteration 90: error = 1.5864296, gradient norm = 0.0194727
[t-SNE] Iterati

We now have put the t-SNE normalized coordinates and document names into the file "data_political_ideologies.json" so they can be used in other environments. This codebase comes with an example (in the `visualize` folder) of displaying these in a webpage using [p5.js](http://www.p5js.org) (javascript). This script uses a simple collision detection procedure to space out all the text so they are not overlapping. 